In [2]:
%config Completer.use_jedi = False
%matplotlib inline
import pandas as pd
pd.options.plotting.backend = "plotly"

## Filter users

Ignore those having orders outside the predicted period

In [3]:
orders = pd.read_csv("../../data/machine_learning_challenge_order_data.csv", parse_dates=["order_date"])

In [4]:
len(orders.customer_id.unique())

245455

In [5]:
bad_users = orders.customer_id[(orders.order_date < "2015-03-01")].unique()

In [6]:
len(bad_users)

17

## Labels

In [7]:
labels = pd.read_csv("../../data/machine_learning_challenge_labeled_data.csv")

In [8]:
labels

,customer_id,is_returning_customer
0,000097eabfd9,0
1,0000e2c6d9be,0
2,000133bb597f,1
3,00018269939b,0
4,0001a00468a6,0
...,...,...
245450,fffd696eaedd,0
245451,fffe9d5a8d41,1
245452,ffff347c3cfa,1
245453,ffff4519b52d,0


In [9]:
labels = labels[~labels.customer_id.isin(bad_users)]

In [10]:
len(labels)

245438

## Features

- n_orders

- order_amount_paid_log10
- order_voucher_amount
- order_voucher_percentage
- order_delivery_fee

    - Aggregations
        - mean
        - med
        - min
        - max

- order_hour
    - counts

- order_date_dayofweek
    - counts

- has_used_voucher
- nused_vouchers
- rate_with_voucher

- has_failed_order
- n_failed_order
- rate_failed_orders

- has_delivery_fee_not_zero
- n_delivery_fee_not_zero
- rate_delivery_fee_not_zero

- days since last order
- gaps between orders: max, avg, median



## TODO:

- Periods:
    - last week
    - last month
    - last 6m
    - last 12m
    - all history



- predominant_city

city and restaurant: use mean encoding (or compute stats: amounts, failed orders, n_orders, etc.)


# Expand raw columns

In [11]:
import numpy as np

In [12]:
df = orders[~orders.customer_id.isin(bad_users)]

In [13]:
df["order_date_dayofweek"] = df.order_date.dt.dayofweek

In [14]:
df["order_amount_paid_log10"] = np.log10(df.amount_paid)

In [15]:
df["order_voucher_notzero"] = df.voucher_amount > 0

In [16]:
df["amount_net"] = df.amount_paid + df.voucher_amount
df["order_voucher_percentage"] = np.round(100 * df.voucher_amount / df.amount_net)

In [17]:
df["order_delivery_fee_not_zero"] = df.delivery_fee > 0

In [18]:
df.columns

Index(['customer_id', 'order_date', 'order_hour', 'customer_order_rank',
       'is_failed', 'voucher_amount', 'delivery_fee', 'amount_paid',
       'restaurant_id', 'city_id', 'payment_id', 'platform_id',
       'transmission_id', 'order_date_dayofweek', 'order_amount_paid_log10',
       'order_voucher_notzero', 'amount_net', 'order_voucher_percentage',
       'order_delivery_fee_not_zero'],
      dtype='object')

# Compute features on full dataset

In [54]:
ds = pd.DataFrame()
ds["customer_id"] = df.customer_id.unique()

In [55]:
ds

,customer_id
0,000097eabfd9
1,0000e2c6d9be
2,000133bb597f
3,00018269939b
4,0001a00468a6
...,...
245433,fffd696eaedd
245434,fffe9d5a8d41
245435,ffff347c3cfa
245436,ffff4519b52d


In [21]:
feats_dfs = []

## Customer order history features

- order_amount_paid_log10
- order_voucher_amount
- order_voucher_percentage
- order_delivery_fee

    - Aggregations
        - mean
        - med
        - min
        - max

In [22]:
aggs = ["mean", "min", "max", "median"]
cols = ['order_amount_paid_log10',
        'voucher_amount',
        'order_voucher_percentage',
        'delivery_fee']

col_aggs = dict([("customer_id", "count")] + [
    (col, aggs) for col in cols
])

feats = df.groupby("customer_id").agg(col_aggs)

feats.columns = ["order__count"] + ['__'.join(col).strip()\
                                    for col in feats.columns.values[1:]]

feats.reset_index(inplace=True)

In [23]:
feats

,customer_id,order__count,order_amount_paid_log10__mean,order_amount_paid_log10__min,order_amount_paid_log10__max,order_amount_paid_log10__median,voucher_amount__mean,voucher_amount__min,voucher_amount__max,voucher_amount__median,order_voucher_percentage__mean,order_voucher_percentage__min,order_voucher_percentage__max,order_voucher_percentage__median,delivery_fee__mean,delivery_fee__min,delivery_fee__max,delivery_fee__median
0,000097eabfd9,1,1.059548,1.059548,1.059548,1.059548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
1,0000e2c6d9be,1,0.980367,0.980367,0.980367,0.980367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
2,000133bb597f,1,0.773536,0.773536,0.773536,0.773536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4930,0.4930,0.4930,0.4930
3,00018269939b,1,0.992266,0.992266,0.992266,0.992266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4930,0.4930,0.4930,0.4930
4,0001a00468a6,1,0.711866,0.711866,0.711866,0.711866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4930,0.4930,0.4930,0.4930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245433,fffd696eaedd,1,1.382628,1.382628,1.382628,1.382628,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4297,1.4297,1.4297,1.4297
245434,fffe9d5a8d41,3,0.995794,0.926492,1.030446,1.030446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
245435,ffff347c3cfa,2,0.827372,0.774313,0.880431,0.827372,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000
245436,ffff4519b52d,1,1.337878,1.337878,1.337878,1.337878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000


In [24]:
feats_dfs.append(feats)

- order_hour
    - counts

- order_date_dayofweek
    - counts

In [25]:
for col in ["order_date_dayofweek", "order_hour"]:
    fdf = df.groupby(["customer_id", col]).size()\
          .unstack(level=1).fillna(0)
    fdf.columns = [f"n_{col}_{h}" for h in fdf.columns]
    
    feats_dfs.append(fdf)

- has_used_voucher
- nused_vouchers
- rate_with_voucher

- has_failed_order
- n_failed_order
- rate_failed_orders

- has_delivery_fee_not_zero
- n_delivery_fee_not_zero
- rate_delivery_fee_not_zero

In [26]:
df["used_voucher"] = (df.voucher_amount > 0).astype(int)
df["failed_order"] = df.is_failed.astype(int)
df["delivery_fee_not_zero"] = (df.delivery_fee > 0).astype(int)

In [27]:
cols = ["used_voucher", "failed_order", "delivery_fee_not_zero"]
aggs = ["max", "sum", "mean"]

col_aggs = dict([
    (col, aggs) for col in cols
])

feats = df.groupby("customer_id").agg(col_aggs)

feats.columns = ['__'.join(col).strip()\
                                    for col in feats.columns.values]

feats.reset_index(inplace=True)

In [28]:
feats

,customer_id,used_voucher__max,used_voucher__sum,used_voucher__mean,failed_order__max,failed_order__sum,failed_order__mean,delivery_fee_not_zero__max,delivery_fee_not_zero__sum,delivery_fee_not_zero__mean
0,000097eabfd9,0,0,0.0,0,0,0.000000,0,0,0.0
1,0000e2c6d9be,0,0,0.0,0,0,0.000000,0,0,0.0
2,000133bb597f,0,0,0.0,0,0,0.000000,1,1,1.0
3,00018269939b,0,0,0.0,0,0,0.000000,1,1,1.0
4,0001a00468a6,0,0,0.0,0,0,0.000000,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...
245433,fffd696eaedd,0,0,0.0,0,0,0.000000,1,1,1.0
245434,fffe9d5a8d41,0,0,0.0,1,2,0.666667,0,0,0.0
245435,ffff347c3cfa,0,0,0.0,0,0,0.000000,0,0,0.0
245436,ffff4519b52d,0,0,0.0,0,0,0.000000,0,0,0.0


In [29]:
feats_dfs.append(feats)

- days since first/last order

In [30]:
from datetime import timedelta

In [31]:
max_date = df.order_date.max() + timedelta(days=1)
max_date

Timestamp('2017-02-28 00:00:00')

In [32]:
r = max_date - df.groupby("customer_id").agg({"order_date": "max"})

r.columns = ["days_since_last_order"]
r.days_since_last_order= r.days_since_last_order.dt.days

feats_dfs.append(r)

In [33]:
r.head()

,days_since_last_order
customer_id,
000097eabfd9,619
0000e2c6d9be,396
000133bb597f,2
00018269939b,23
0001a00468a6,574


In [34]:
r = max_date - df.groupby("customer_id").agg({"order_date": "min"})

r.columns = ["days_since_first_order"]
r.days_since_first_order= r.days_since_first_order.dt.days

feats_dfs.append(r)

In [35]:
r.head()

,days_since_first_order
customer_id,
000097eabfd9,619
0000e2c6d9be,396
000133bb597f,2
00018269939b,23
0001a00468a6,574


**TODO**:
- gaps between orders: max, avg, median

- Issues:
  it only makes sense for customers with >1 order. We have to figure out how to handle missing values

## Join all feature dataframes

In [56]:
for fdf in feats_dfs + [labels]:
    ds = pd.merge(ds, fdf,
              how="inner", on="customer_id")

In [57]:
ds

,customer_id,order__count,order_amount_paid_log10__mean,order_amount_paid_log10__min,order_amount_paid_log10__max,order_amount_paid_log10__median,voucher_amount__mean,voucher_amount__min,voucher_amount__max,voucher_amount__median,...,used_voucher__mean,failed_order__max,failed_order__sum,failed_order__mean,delivery_fee_not_zero__max,delivery_fee_not_zero__sum,delivery_fee_not_zero__mean,days_since_last_order,days_since_first_order,is_returning_customer
0,000097eabfd9,1,1.059548,1.059548,1.059548,1.059548,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,0,0,0.0,619,619,0
1,0000e2c6d9be,1,0.980367,0.980367,0.980367,0.980367,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,0,0,0.0,396,396,0
2,000133bb597f,1,0.773536,0.773536,0.773536,0.773536,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,1,1,1.0,2,2,1
3,00018269939b,1,0.992266,0.992266,0.992266,0.992266,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,1,1,1.0,23,23,0
4,0001a00468a6,1,0.711866,0.711866,0.711866,0.711866,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,1,1,1.0,574,574,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245433,fffd696eaedd,1,1.382628,1.382628,1.382628,1.382628,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,1,1,1.0,533,533,0
245434,fffe9d5a8d41,3,0.995794,0.926492,1.030446,1.030446,0.0,0.0,0.0,0.0,...,0.0,1,2,0.666667,0,0,0.0,151,212,1
245435,ffff347c3cfa,2,0.827372,0.774313,0.880431,0.827372,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,0,0,0.0,166,195,1
245436,ffff4519b52d,1,1.337878,1.337878,1.337878,1.337878,0.0,0.0,0.0,0.0,...,0.0,0,0,0.000000,0,0,0.0,332,332,0


In [58]:
for f in ds.columns:
    print(f)

customer_id
order__count
order_amount_paid_log10__mean
order_amount_paid_log10__min
order_amount_paid_log10__max
order_amount_paid_log10__median
voucher_amount__mean
voucher_amount__min
voucher_amount__max
voucher_amount__median
order_voucher_percentage__mean
order_voucher_percentage__min
order_voucher_percentage__max
order_voucher_percentage__median
delivery_fee__mean
delivery_fee__min
delivery_fee__max
delivery_fee__median
n_order_date_dayofweek_0
n_order_date_dayofweek_1
n_order_date_dayofweek_2
n_order_date_dayofweek_3
n_order_date_dayofweek_4
n_order_date_dayofweek_5
n_order_date_dayofweek_6
n_order_hour_0
n_order_hour_1
n_order_hour_2
n_order_hour_3
n_order_hour_4
n_order_hour_5
n_order_hour_6
n_order_hour_7
n_order_hour_8
n_order_hour_9
n_order_hour_10
n_order_hour_11
n_order_hour_12
n_order_hour_13
n_order_hour_14
n_order_hour_15
n_order_hour_16
n_order_hour_17
n_order_hour_18
n_order_hour_19
n_order_hour_20
n_order_hour_21
n_order_hour_22
n_order_hour_23
used_voucher__max
used

## City features

**TODO** : Compute city features using full dataset, then aggregate city features over orders of each customer_id (e.g.: City total customers, city total orders, avg spend on city, etc.)

## Restaurant features


**TODO** : Compute restaurant features, similarly to city features.

**TODO** Replicate same feature sets but limited to recent time windows (last month, last 6 months, etc.)

## Add split column
70/10/20 train/val/test split

In [60]:
from random import random

In [61]:
def get_split(c):
    r = random()
    if r < 0.7:
        return "train"
    elif r < 0.8:
        return "val"
    else:
        return "test"

In [62]:
ds["split"] = ds.customer_id.apply(get_split)

In [63]:
ds.groupby("split").size()/len(ds)

split
test     0.200055
train    0.699696
val      0.100249
dtype: float64

## Save dataset to file

In [72]:
ds.shape

(245438, 62)

In [68]:
ds.to_csv("../../data/dataset.csv", index=False)

In [69]:
r = pd.read_csv("../../data/dataset.csv")

In [70]:
r.shape

(245438, 62)

In [71]:
r.head()

,customer_id,order__count,order_amount_paid_log10__mean,order_amount_paid_log10__min,order_amount_paid_log10__max,order_amount_paid_log10__median,voucher_amount__mean,voucher_amount__min,voucher_amount__max,voucher_amount__median,...,failed_order__max,failed_order__sum,failed_order__mean,delivery_fee_not_zero__max,delivery_fee_not_zero__sum,delivery_fee_not_zero__mean,days_since_last_order,days_since_first_order,is_returning_customer,split
0,000097eabfd9,1,1.059548,1.059548,1.059548,1.059548,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,0.0,619,619,0,test
1,0000e2c6d9be,1,0.980367,0.980367,0.980367,0.980367,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,0.0,396,396,0,test
2,000133bb597f,1,0.773536,0.773536,0.773536,0.773536,0.0,0.0,0.0,0.0,...,0,0,0.0,1,1,1.0,2,2,1,train
3,00018269939b,1,0.992266,0.992266,0.992266,0.992266,0.0,0.0,0.0,0.0,...,0,0,0.0,1,1,1.0,23,23,0,train
4,0001a00468a6,1,0.711866,0.711866,0.711866,0.711866,0.0,0.0,0.0,0.0,...,0,0,0.0,1,1,1.0,574,574,0,train
